In [3]:
import pandas as pd

posts = pd.read_csv('post_img_polarity.csv.gz')
len(posts), posts.columns

(17826,
 Index(['Unnamed: 0', 'Unnamed: 0.1', 'funct', 'pronoun', 'ppron', 'i', 'we',
        'you', 'shehe', 'they', 'ipron', 'article', 'verb', 'auxverb', 'past',
        'present', 'future', 'adverb', 'preps', 'conj', 'negate', 'quant',
        'number', 'swear', 'social', 'family', 'friend', 'humans', 'affect',
        'posemo', 'negemo', 'anx', 'anger', 'sad', 'cogmech', 'insight',
        'cause', 'discrep', 'tentat', 'certain', 'inhib', 'incl', 'excl',
        'percept', 'see', 'hear', 'feel', 'bio', 'body', 'health', 'sexual',
        'ingest', 'relativ', 'motion', 'space', 'time', 'work', 'achieve',
        'leisure', 'home', 'money', 'relig', 'death', 'assent', 'nonfl',
        'filler', 'post.id', 'blog.id', 'polarity', 'img_polarity', 'img'],
       dtype='object'))

In [9]:
def polarity(post):
    if post['polarity'] > 3:
        return 0
    if post['polarity'] < -3:
        return 2
    else:
        return 1

posts['post_polarity'] = posts.apply(polarity, axis=1)

posts[['post_polarity','img_polarity']].tail(10)

,post_polarity,img_polarity
17816,0,0.0
17817,0,1.0
17818,1,1.0
17819,0,2.0
17820,0,2.0
17821,0,2.0
17822,0,1.0
17823,1,1.0
17824,1,0.0
17825,1,1.0


In [14]:
from sklearn.metrics import accuracy_score

accuracy_score(posts['post_polarity'].values.astype(int), posts['img_polarity'].values.astype(int))

0.378099405362953